In [2]:
import requests
import pandas as pd
import subprocess
import time
from fastapi import FastAPI
from fastapi.responses import JSONResponse
import json
import tensorflow as tf
from tokenization import tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gsevr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
def all_articles():
    global df

    # lists for df
    title = []
    date = []
    url = []
    prediction = []

    api = 'b456a0b678604587bd37dfe28a0bf520'

    base_url = 'https://newsapi.org/v2/everything'

    params = {
        'source': 'bbc-news',
        'q': '+good',
        'language': 'en',
        'sortBy': 'publishedAt',
        'apiKey': api
    }

    gathered_articles = requests.get(base_url, params=params)

    data = gathered_articles.json()

    articles = data['articles']


    # fetch sentiment analysis results
    model = tf.keras.models.load_model('new.h5')

    for article in articles[:100]:
        txt_seq = tokenizer.texts_to_sequences([article['content']])
        pad_seq = pad_sequences(txt_seq, maxlen=100, padding='post')
        pred = model.predict(pad_seq)
        if np.argmax(pred) == 0:
            if article['title'] != '[Removed]':
                title.append(article['title'])
                date.append(article['publishedAt'])
                url.append(article['url'])
                prediction.append(pred[np.argmax(pred)][0])

            # responses.append(article)
        


    # for response in responses:
    #     if response['title'] != '[Removed]':
    #         title.append(response['title'])
    #         date.append(response['publishedAt'])
    #         url.append(response['url'])


    df = pd.DataFrame([title,date,url,prediction]).T
    df.columns = ['title','date','url','prediction']
    # df = df.sort_values(by=prediction)
    df.to_excel('articles.xlsx')

    df_json = json.loads(df.head(10).to_json(orient='records'))
    return df
    

all_articles()

1/1 [==============================] - 0s 18ms/step


,title,date,url,prediction
0,WhatsApp is testing a feature to secure your l...,2023-10-10T18:45:17Z,https://www.androidcentral.com/apps-software/w...,0.99577
1,‘Squealer’ Red Band Trailer – Tyrese Gibson St...,2023-10-10T18:44:49Z,https://bloody-disgusting.com/movie/3783260/sq...,0.999969
2,As Manufacturing Returns Home - Here Is How AI...,2023-10-10T18:44:40Z,https://www.forbes.com/sites/paulnoble/2023/10...,0.999704
3,Commandos: Origins revives the WW2 real-time s...,2023-10-10T18:43:42Z,https://www.eurogamer.net/commandos-origins-re...,0.999319
4,CCI to firm up rules for leniency-plus regime:...,2023-10-10T18:43:25Z,https://economictimes.indiatimes.com/news/econ...,0.998574
...,...,...,...,...
86,3 Top Stocks to Buy with $50,2023-10-10T18:18:49Z,https://biztoc.com/x/fbc15302143f5228,0.995088
87,Cyberpunk 2077: Phantom Liberty,2023-10-10T18:18:30Z,https://me.pcmag.com/en/microsoft-xbox-games/1...,0.999973
88,Cyberpunk 2077: Phantom Liberty,2023-10-10T18:18:30Z,https://uk.pcmag.com/microsoft-xbox-games/1490...,0.999973
89,The Kid Laroi Is Going Global On His Next Sing...,2023-10-10T18:17:48Z,https://uproxx.com/pop/the-kid-laroi-too-much-...,0.99696


In [11]:
df.sort_values(by='prediction')

,title,date,url,prediction
64,Why Wells Fargo sees potentially ‘good entry p...,2023-10-10T18:24:07Z,https://biztoc.com/x/f1c425ee00721063,0.537658
62,"Playtex Sport Tampons, Multipack, Fragrance-Fr...",2023-10-10T18:24:59Z,https://queenbeetoday.com/playtex-sport-tampon...,0.546461
21,Documenting Journey Into AI Safety,2023-10-10T18:37:14Z,https://www.lesswrong.com/posts/ozDWnEChJwuB5L...,0.657327
81,#EmacsConf backstage: file prefixes,2023-10-10T18:20:08Z,https://sachachua.com/blog/2023/10/emacsconf-b...,0.666482
30,This Graphic Designer–Turned–Cabinetmaker’s Dy...,2023-10-10T18:34:02Z,https://www.sightunseen.com/2023/10/jonathan-c...,0.697375
...,...,...,...,...
75,NFL Week 6 Expert Picks & Predictions: Emory H...,2023-10-10T18:21:20Z,https://thesportsdaily.com/news/nfl-week-6-exp...,0.999976
40,Amazfit's cheap GTS smartwatches are up to $50...,2023-10-10T18:30:29Z,https://www.androidpolice.com/amazfit-gts4-gts...,0.999984
18,Mayo Clinic Q&A: Foods to help maximize your m...,2023-10-10T18:38:35Z,https://www.bostonherald.com/2023/10/10/mayo-c...,0.999995
11,Bill Cowher dismisses Deion Sanders' idea for ...,2023-10-10T18:41:25Z,https://www.marca.com/en/nfl/2023/10/10/652598...,0.999996
